In [1]:
import pandas as pd 
import math
import numpy as np
import tensorflow as tf
#from tqdm import tqdm_notebook as tqdm # Jupyter notebook should use this
import matplotlib.pyplot as plt
import os
import cv2
import scipy.io as scipy_io
os.environ["CUDA_VISIBLE_DEVICES"]="3"; 
train_dataset = np.array(pd.read_csv("UCI dataset/mushroom", header = None,delimiter=' '))
train_dataset.shape

label = train_dataset[:,0]
data = train_dataset[:,1:22]
label = label.reshape(label.shape[0], 1)
print(label)

tr_file = 'UCI dataset/mushroom'
INPUT_DIMENSION = 112
NUM_CLASSES = 2

def process_data(fname, NUM_ATTR, NUM_CLASSES):
    I = np.eye(NUM_CLASSES)
    with open(fname) as file:
        xx = file.readlines()
        #print(xx)
        data = np.zeros([len(xx), NUM_ATTR])
        label = np.zeros(len(xx), dtype=int)
        label_onehot = []
        for i in range(len(xx)):
            tmp = xx[i].split(' ')
            label[i] = int(tmp[0])-1
            label_onehot.append(I[label[i]])
            for j in range(1,len(tmp)-1):
                position = int(tmp[j].split(':')[0])
                value = 1.0*int(tmp[j].split(':')[1])
                data[i][position-1] = value 
    return data, label, np.array(label_onehot, dtype=int)

/usr/lib/python3/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


[[1]
 [2]
 [2]
 ...
 [2]
 [1]
 [2]]


In [2]:
tr_data, tr_label, tr_onehot = process_data(tr_file, NUM_ATTR=112, NUM_CLASSES=2)


In [3]:
y_train = tr_label[:4000]
y_test = tr_label[4000:]

tr_data = tr_data*0.01
x_train = tr_data[:4000]
x_test = tr_data[4000:]
y_train_onehot = tr_onehot[:4000]
y_test_onehot = tr_onehot[4000:]
print(x_train)
print(x_test.shape)
print(y_train_onehot.shape)
print(y_test_onehot.shape)
INPUT_DIMENSION = 112
NUM_CLASSES = 2

[[0.   0.   0.   ... 0.   0.01 0.  ]
 [0.   0.   0.   ... 0.   0.   0.  ]
 [0.   0.01 0.   ... 0.   0.   0.  ]
 ...
 [0.   0.   0.   ... 0.   0.   0.  ]
 [0.   0.   0.01 ... 0.   0.   0.  ]
 [0.   0.   0.01 ... 0.   0.   0.  ]]
(4124, 112)
(4000, 2)
(4124, 2)


np.savetxt("x_train.csv", x_train, delimiter=",")
np.savetxt("y_train.csv", y_train, delimiter=",")
np.savetxt("x_test.csv", x_test, delimiter=",")
np.savetxt("y_test.csv", y_test, delimiter=",")

In [4]:

border = 400
x_train_init = x_train[:border]
y_train_init = y_train[:border]
x_train_seq = x_train[border:]
y_train_seq = y_train[border:]

import time
from sklearn.decomposition import PCA
# Make an instance of the Model
pca = PCA(0.55)
time_sum = 0
t1 = time.time()
pca.fit(x_train)
t2 = time.time()
time_sum+= t2-t1
train_img = pca.transform(x_train)
test_img  = pca.transform(x_test)
x_train.shape,train_img.shape

In [5]:
x_train.shape,y_train.shape

((4000, 112), (4000,))

border = 146
x_train_init = train_img[:border]
y_train_init = y_train[:border]
x_train_seq = train_img[border:]
y_train_seq = y_train[border:]

In [6]:
tf.reset_default_graph() # clear all the tensors
sess = tf.Session()


"""Placeholders"""
X_ = tf.placeholder(tf.float32, [None, INPUT_DIMENSION])
#X_ = tf.reshape(X, [-1, INPUT_DIMENSION]) # Flatten X: [N,D]
Y = tf.placeholder(tf.int64, [None]) # labels
Y_ = tf.one_hot(indices=Y, depth=NUM_CLASSES) # one_hot labels: [N,M]

"""Some constants"""
D = INPUT_DIMENSION
M = NUM_CLASSES # Number of outputs
C = tf.constant(2.0**(-3))

"""Weights"""
alpha_1 = tf.get_variable('alpha_1',shape=[D, M],initializer=tf.random_uniform_initializer(-1,1),trainable=False) # 1st subnetwork
alpha_2 = tf.get_variable('alpha_2',shape=[D, M],initializer=tf.random_uniform_initializer(-1,1),trainable=False) # 2st subnetwork
alpha_3 = tf.get_variable('alpha_3',shape=[D, M],initializer=tf.random_uniform_initializer(-1,1),trainable=False)
beta_1 = tf.get_variable('beta_1',shape=[M, M],initializer=tf.random_uniform_initializer(-1,1),trainable=False)
beta_2 = tf.get_variable('beta_2',shape=[M, M],initializer=tf.random_uniform_initializer(-1,1),trainable=False)
beta_3 = tf.get_variable('beta_3',shape=[M, M],initializer=tf.random_uniform_initializer(-1,1),trainable=False)

In [7]:
k = tf.get_variable('k',shape=[D, D],initializer=tf.random_uniform_initializer(-1,1),trainable=False)
m = tf.get_variable('m',shape=[M, M],initializer=tf.random_uniform_initializer(-1,1),trainable=False)

In [8]:
"""Utility functions"""
def mul(A, B):
    return tf.matmul(A, B)

def inv(A):
    return tf.matrix_inverse(A)

def t(A):
    return tf.transpose(A)

def sin(A):
    return tf.math.sin(A)

def asin(A):
    return tf.math.asin(A)

def sqrt(A):
    return tf.sqrt(A)

def sqr(A):
    return tf.math.pow(A, 2)

def pseudo_inv(A, I, C):
    C_I = I/C
    return mul(t(A), inv(C_I + mul(A, t(A))))

def h(A):
    '''activation function'''
    return sin(A)

def h_(A):
    '''inverse activation function'''
    return asin(A)

def u(A):
    '''normalize the input to (0,1]'''
    return tf.math.sigmoid(A) # sigmoid
    
def u_(A):
    '''the inverse of u'''
    ONE = tf.constant(1.0)
    return -(tf.math.log(ONE/A - ONE)) # the inverse of sigmoid
    
def subnet_output(alpha, beta, A):
    return t(mul(beta, h(mul(t(alpha), t(A))))) 

# Initial Training

In [9]:
"""Initial Training Graph"""
# D: input dimension
# N: number of input samples
# M: number of classes (number of outputs)
'''some pre-computations'''
X_init = t(X_) # [D,N]
Y_init = t(Y_) # [M,N]
N_init = D # number of dimensions
I_DxD = tf.eye(N_init, dtype=tf.float32) # [D,D]
I_MxM = tf.eye(M, dtype=tf.float32) # [M,M]
C_I = I_DxD/C
H_I = I_MxM/C

add = C_I + mul(X_init, t(X_init))
k = tf.assign(k,add)
X_inv_init = pseudo_inv(X_init, I_DxD, C) # [N,D]

'''1st subnet'''
alpha_1_init_calculated = t(mul(h_(Y_init), X_inv_init)) # ([M,N]x[N,D])T=[D,M]
alpha_1_init = tf.assign(alpha_1, alpha_1_init_calculated) # [D,M]
H_1_init = h(mul(t(alpha_1_init), X_init)) # [M,N]
H_add = H_I + mul(H_1_init,t(H_1_init))
m = tf.assign(m,H_add)
H_pseudo_init = pseudo_inv(H_1_init,I_MxM,C) #[N,M]

beta_1_init_calculated = mul(Y_init, t(H_1_init))/sqr(tf.norm(H_1_init)) # [M,M]
beta_1_init_calculated = mul(Y_init,H_pseudo_init)

beta_1_init = tf.assign(beta_1, beta_1_init_calculated) # [M,M]
H_beta_1_init = mul(beta_1_init, t(mul(t(X_init), alpha_1_init))) # [M,N]
E_1_init = Y_init - H_beta_1_init # [M,N]

'''2nd subnet'''
#alpha_2_init_calculated = t(mul(h_(E_1_init), X_inv_init)) # [D,M]    
alpha_2_init_calculated = t(mul(asin(Y_init), X_inv_init)) # [D,M]

alpha_2_init = tf.assign(alpha_2, alpha_2_init_calculated) # [D,M]
H_2_init = h(mul(t(alpha_2_init), X_init)) # [M,N]
H_2_inv_init = pseudo_inv(H_2_init, I_MxM, C) # [M,N]
H_add = H_I + mul(H_2_init,t(H_2_init))
#m = tf.assign(m,H_add)
H_pseudo_init = pseudo_inv(H_2_init,I_MxM,C) #[N,M]

beta_2_init_calculated = mul(E_1_init, t(H_2_init))/sqr(tf.norm(H_2_init)) # [M,M]
beta_2_init_calculated = mul(E_1_init, H_pseudo_init)

beta_2_init = tf.assign(beta_2, beta_2_init_calculated) # [M,M]
H_beta_2_init = mul(beta_2_init, t(mul(t(X_init), alpha_2_init))) # [M,N]
E_2_init = Y_init - (H_beta_1_init+H_beta_2_init) # [M,N]

'''3rd subnetwork'''
alpha_3_init_calculated = t(mul(h_(E_2_init), X_inv_init)) # [D,M]    
alpha_3_init_calculated = t(mul(asin(Y_init), X_inv_init)) # [D,M]

alpha_3_init = tf.assign(alpha_3, alpha_3_init_calculated) # [D,M]
H_3_init = h(mul(t(alpha_3_init), X_init)) # [M,N]
H_3_inv_init = pseudo_inv(H_3_init, I_MxM, C) # [M,N]

beta_3_init_calculated = mul(E_2_init, t(H_3_init))/sqr(tf.norm(H_3_init)) # [M,M]
beta_3_init_calculated = mul(E_2_init, H_3_inv_init)

beta_3_init = tf.assign(beta_3, beta_3_init_calculated) # [M,M]
H_beta_3_init = mul(beta_3_init, t(mul(t(X_init), alpha_3_init))) # [M,N]
E_3_init = Y_init - (H_beta_3_init+H_beta_2_init+ H_beta_1_init) # [M,N]

#init_train_graph = H_beta_1_init
init_train_graph = E_3_init



In [10]:
beta_1_init

<tf.Tensor 'Assign_3:0' shape=(2, 2) dtype=float32_ref>

In [11]:
"""Evaluate initial-training graph"""
#logits_init =  subnet_output(alpha_1, beta_1, X_)
logits_init = subnet_output(alpha_1, beta_1, X_) + subnet_output(alpha_2, beta_2, X_) + subnet_output(alpha_3, beta_3, X_)
loss_init = tf.losses.mean_squared_error(labels=Y_, predictions=logits_init)
accuracy_init = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(Y_, axis=1), tf.argmax(logits_init, axis=1)), dtype=tf.float32))

In [12]:

'''With one subnetwork'''
logits_init = subnet_output(alpha_1, beta_1, X_) 
loss_init = tf.losses.mean_squared_error(labels=Y_, predictions=logits_init)
accuracy_init = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(Y_, axis=1), tf.argmax(logits_init, axis=1)), dtype=tf.float32))

In [13]:
"""Initialize variables"""
init = tf.global_variables_initializer()
sess.run(init)

In [14]:
"""Initial training"""
sess.run(E_1_init, feed_dict={X_: x_train_init, Y: y_train_init})
print("Initial training done")

"""Initial training evaluation"""
tr_loss, tr_acc = sess.run([loss_init, accuracy_init], feed_dict={X_: x_train, Y: y_train})
ts_loss, ts_acc = sess.run([loss_init, accuracy_init], feed_dict={X_: x_test, Y: y_test})
print("Initial train training loss: ", tr_loss)
print("Initial train training accuracy: ", tr_acc)
print("Initial train testing loss: ", ts_loss)
print("Initial train testing accuracy: ", ts_acc)

Initial training done
Initial train training loss:  0.3311794
Initial train training accuracy:  0.82725
Initial train testing loss:  0.45841968
Initial train testing accuracy:  0.21799225


In [15]:
"""Evaluate initial-training graph"""
#logits_init =  subnet_output(alpha_1, beta_1, X_)
logits_init = subnet_output(alpha_1, beta_1, X_) + subnet_output(alpha_2, beta_2, X_) 
loss_init = tf.losses.mean_squared_error(labels=Y_, predictions=logits_init)
accuracy_init = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(Y_, axis=1), tf.argmax(logits_init, axis=1)), dtype=tf.float32))

In [16]:
"""Initial training"""
import time
s = 0
t1 = time.time()
sess.run(E_2_init, feed_dict={X_: x_train_init, Y: y_train_init})
t2 = time.time()
s+=t2-t2
print("Initial training done")

"""Initial training evaluation"""
tr_loss, tr_acc = sess.run([loss_init, accuracy_init], feed_dict={X_: x_train, Y: y_train})
ts_loss, ts_acc = sess.run([loss_init, accuracy_init], feed_dict={X_: x_test, Y: y_test})
print("Initial train training loss: ", tr_loss)
print("Initial train training accuracy: ", tr_acc)
print("Initial train testing loss: ", ts_loss)
print("Initial train testing accuracy: ", ts_acc)

Initial training done
Initial train training loss:  0.2444909
Initial train training accuracy:  0.82725
Initial train testing loss:  0.44731468
Initial train testing accuracy:  0.21799225


In [17]:
"""Evaluate initial-training graph"""
#logits_init =  subnet_output(alpha_1, beta_1, X_)
logits_init = subnet_output(alpha_1, beta_1, X_) + subnet_output(alpha_2, beta_2, X_) + subnet_output(alpha_3, beta_3, X_)
loss_init = tf.losses.mean_squared_error(labels=Y_, predictions=logits_init)
accuracy_init = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(Y_, axis=1), tf.argmax(logits_init, axis=1)), dtype=tf.float32))

In [18]:
"""Initial training"""
sess.run(init_train_graph, feed_dict={X_: x_train_init, Y: y_train_init})
print("Initial training done")

"""Initial training evaluation"""
tr_loss, tr_acc = sess.run([loss_init, accuracy_init], feed_dict={X_: x_train, Y: y_train})
ts_loss, ts_acc = sess.run([loss_init, accuracy_init], feed_dict={X_: x_test, Y: y_test})
print("Initial train training loss: ", tr_loss)
print("Initial train training accuracy: ", tr_acc)
print("Initial train testing loss: ", ts_loss)
print("Initial train testing accuracy: ", ts_acc)

Initial training done
Initial train training loss:  0.19943665
Initial train training accuracy:  0.82725
Initial train testing loss:  0.44852966
Initial train testing accuracy:  0.21799225


In [19]:
logits_init = subnet_output(alpha_1, beta_1, X_)+ subnet_output(alpha_2, beta_2, X_)

In [20]:
logic = sess.run(logits_init, feed_dict={X_ : [x_test[4]]})
print(np.argmax(logic,axis =1))
print(y_test[4])
#plt.imshow(x_test[4200])

[1]
0


# Sequential Training

In [21]:
"""Sequential Training Graph"""
# D: input dimension
# N: number of input samples
# M: number of classes (number of outputs)
X_seq = t(X_) # [D,N]
Y_seq = t(Y_) # [M,N]
pseudo = mul(X_seq, X_) #DXD
k = tf.assign(k, tf.add(k,pseudo)) #DXD
k_inv = inv(k)

new = tf.matmul(tf.matmul(k_inv, X_seq),h_(Y_) - tf.matmul(X_, alpha_1))
alpha1_seq = tf.assign(alpha_1,tf.add(alpha_1,new)) #DXM
H_1_seq = h(mul(t(alpha1_seq), X_seq)) # [M,N]
m_su = mul(H_1_seq,t(H_1_seq))
m = tf.assign(m,tf.add(m,m_su))
m_inv = inv(m)
#update = tf.matmul(tf.matmul(m_inv,H_1_seq),h_(Y_seq)- tf.matmul())
H_pseudo_init = pseudo_inv(H_1_seq,I_MxM,C) #[N,M]
#UPDATE = tf.matmul(tf.matmul(K_inverse, HT), inverse_acti_y - tf.matmul(H, self.__outputWeight))
beta_1_seq_calculated = mul(Y_seq, H_pseudo_init) # [M,M]
beta_1_seq = tf.assign(beta_1, beta_1_seq_calculated) # [M,M]
H_beta_1_seq = mul(beta_1_seq, t(mul(X_, alpha1_seq))) # [M,N]
E_1_seq = Y_seq - H_beta_1_seq # [M,N]

'''2nd subnetwork'''

new = tf.matmul(tf.matmul(k_inv, X_seq),h_(Y_) - tf.matmul(X_, alpha_2))
alpha2_seq = tf.assign(alpha_2,tf.add(alpha_2,new)) #DXM
H_2_seq = h(mul(t(alpha2_seq), X_seq)) # [M,N]
H_pseudo_init = pseudo_inv(H_2_seq,I_MxM,C) #[N,M]
beta_2_seq_calculated = mul(E_1_seq, H_pseudo_init) # [M,M]
beta_2_seq = tf.assign(beta_2, beta_2_seq_calculated) # [M,M]
H_beta_2_seq = mul(beta_2_seq, t(mul(t(X_seq), alpha2_seq))) # [M,N]
E_2_seq = Y_seq - (H_beta_2_seq+ H_beta_1_seq) # [M,N]

'''3rd subnetwork'''
new = tf.matmul(tf.matmul(k_inv, X_seq),h_(Y_) - tf.matmul(X_, alpha_3))
alpha3_seq = tf.assign(alpha_3,tf.add(alpha_3,new)) #DXM
H_3_seq = h(mul(t(alpha3_seq), X_seq)) # [M,N]
H_pseudo_init = pseudo_inv(H_3_seq,I_MxM,C) #[N,M]
beta_3_seq_calculated = mul(E_2_seq, H_pseudo_init) # [M,M]
beta_3_seq = tf.assign(beta_3, beta_3_seq_calculated) # [M,M]
H_beta_3_seq = mul(beta_3_seq, t(mul(t(X_seq), alpha3_seq))) # [M,N]
E_3_seq = Y_seq - (H_beta_3_seq +H_beta_2_seq + H_beta_1_seq )# [M,N]
seq_train_graph = E_3_seq




In [22]:
"""Evaluate initial-training graph"""
logits_seq =  subnet_output(alpha_1, beta_1, X_) + subnet_output(alpha_2, beta_2, X_) + subnet_output(alpha_3, beta_3, X_)
#logits_init = subnet_output(alpha_1, beta_1, X_) + subnet_output(alpha_2, beta_2, X_)
loss_seq = tf.losses.mean_squared_error(labels=Y_, predictions=logits_seq)
accuracy_seq = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(Y_, axis=1), tf.argmax(logits_seq, axis=1)), dtype=tf.float32))

In [23]:
"""Sequential training"""
batch_size = 100

epoch_train_accuracy = []
epoch_test_accuracy = []
t1 = time.time()
for epoch in range(705):
    #pbar = tqdm.tqdm(total=len(x_train), desc='sequential training phase')
    for i in range(0, len(x_train_seq), batch_size):
        x_batch = x_train_seq[i:i+batch_size]
        y_batch = y_train_seq[i:i+batch_size]
        if len(x_batch) != batch_size:
            break
        sess.run(seq_train_graph, feed_dict={X_: x_batch, Y: y_batch})
        #pbar.update(n=len(x_batch))
    '''epoch evaluation'''
    [train_loss, train_accuracy] = sess.run([loss_seq, accuracy_seq], feed_dict={X_: x_train, Y: y_train})
    [test_loss, test_accuracy] = sess.run([loss_seq, accuracy_seq], feed_dict={X_: x_test, Y: y_test})
    print('train_loss: %f, train_accuracy: %f' % (train_loss, train_accuracy))
    print('test_loss: %f, test_accuracy: %f' % (test_loss, test_accuracy))
    epoch_train_accuracy.append(train_accuracy)
    epoch_test_accuracy.append(test_accuracy)
#sess.run(init_train_graph, feed_dict={X: x_train_init, Y: y_train_init})
t2 = time.time()
s+=(t2-t1)/705
print("Sequential training done")

"""Sequential training evaluation"""
#tr_loss, tr_acc = sess.run([loss_init, accuracy_init], feed_dict={X: x_train, Y: y_train})
#ts_loss, ts_acc = sess.run([loss_init, accuracy_init], feed_dict={X: x_test, Y: y_test})
#print("Sequential train training loss: ", tr_loss)
#print("Sequential train training accuracy: ", tr_acc)
#print("Sequential train testing loss: ", ts_loss)
#print("Sequential train testing accuracy: ", ts_acc)

train_loss: 0.433832, train_accuracy: 0.172750
test_loss: 0.192498, test_accuracy: 0.782008
train_loss: 0.380313, train_accuracy: 0.172750
test_loss: 0.180496, test_accuracy: 0.782008
train_loss: 0.329648, train_accuracy: 0.172750
test_loss: 0.169515, test_accuracy: 0.782008
train_loss: 0.285872, train_accuracy: 0.185500
test_loss: 0.160452, test_accuracy: 0.783463
train_loss: 0.248746, train_accuracy: 0.540250
test_loss: 0.153814, test_accuracy: 0.802861
train_loss: 0.218575, train_accuracy: 0.753500
test_loss: 0.149407, test_accuracy: 0.830262
train_loss: 0.195069, train_accuracy: 0.775500
test_loss: 0.146696, test_accuracy: 0.844083
train_loss: 0.177257, train_accuracy: 0.780000
test_loss: 0.145119, test_accuracy: 0.847721
train_loss: 0.163924, train_accuracy: 0.786500
test_loss: 0.144240, test_accuracy: 0.850873
train_loss: 0.153938, train_accuracy: 0.793500
test_loss: 0.143770, test_accuracy: 0.856208
train_loss: 0.146394, train_accuracy: 0.813500
test_loss: 0.143536, test_accurac

train_loss: 0.098128, train_accuracy: 0.975250
test_loss: 0.137163, test_accuracy: 0.863725
train_loss: 0.097998, train_accuracy: 0.975250
test_loss: 0.137018, test_accuracy: 0.863725
train_loss: 0.097870, train_accuracy: 0.975250
test_loss: 0.136875, test_accuracy: 0.863725
train_loss: 0.097745, train_accuracy: 0.975250
test_loss: 0.136732, test_accuracy: 0.863967
train_loss: 0.097621, train_accuracy: 0.975250
test_loss: 0.136591, test_accuracy: 0.864210
train_loss: 0.097500, train_accuracy: 0.975250
test_loss: 0.136451, test_accuracy: 0.864452
train_loss: 0.097380, train_accuracy: 0.975250
test_loss: 0.136312, test_accuracy: 0.864937
train_loss: 0.097263, train_accuracy: 0.975250
test_loss: 0.136175, test_accuracy: 0.865179
train_loss: 0.097147, train_accuracy: 0.975250
test_loss: 0.136039, test_accuracy: 0.865179
train_loss: 0.097033, train_accuracy: 0.975250
test_loss: 0.135904, test_accuracy: 0.865664
train_loss: 0.096921, train_accuracy: 0.975250
test_loss: 0.135770, test_accurac

train_loss: 0.091695, train_accuracy: 0.975250
test_loss: 0.128759, test_accuracy: 0.883366
train_loss: 0.091657, train_accuracy: 0.975250
test_loss: 0.128706, test_accuracy: 0.883123
train_loss: 0.091619, train_accuracy: 0.975250
test_loss: 0.128653, test_accuracy: 0.883123
train_loss: 0.091581, train_accuracy: 0.975250
test_loss: 0.128601, test_accuracy: 0.883123
train_loss: 0.091543, train_accuracy: 0.975250
test_loss: 0.128550, test_accuracy: 0.883123
train_loss: 0.091506, train_accuracy: 0.975250
test_loss: 0.128499, test_accuracy: 0.883123
train_loss: 0.091469, train_accuracy: 0.975250
test_loss: 0.128448, test_accuracy: 0.883123
train_loss: 0.091433, train_accuracy: 0.975250
test_loss: 0.128398, test_accuracy: 0.883366
train_loss: 0.091397, train_accuracy: 0.975250
test_loss: 0.128349, test_accuracy: 0.883366
train_loss: 0.091361, train_accuracy: 0.975250
test_loss: 0.128300, test_accuracy: 0.883366
train_loss: 0.091325, train_accuracy: 0.975250
test_loss: 0.128252, test_accurac

train_loss: 0.089043, train_accuracy: 0.975250
test_loss: 0.125606, test_accuracy: 0.885063
train_loss: 0.089020, train_accuracy: 0.975250
test_loss: 0.125585, test_accuracy: 0.885063
train_loss: 0.088996, train_accuracy: 0.975250
test_loss: 0.125564, test_accuracy: 0.885063
train_loss: 0.088973, train_accuracy: 0.975250
test_loss: 0.125543, test_accuracy: 0.885063
train_loss: 0.088949, train_accuracy: 0.975250
test_loss: 0.125522, test_accuracy: 0.885063
train_loss: 0.088926, train_accuracy: 0.975250
test_loss: 0.125502, test_accuracy: 0.885063
train_loss: 0.088903, train_accuracy: 0.975250
test_loss: 0.125481, test_accuracy: 0.885306
train_loss: 0.088880, train_accuracy: 0.975250
test_loss: 0.125461, test_accuracy: 0.885306
train_loss: 0.088856, train_accuracy: 0.975250
test_loss: 0.125441, test_accuracy: 0.885306
train_loss: 0.088833, train_accuracy: 0.975250
test_loss: 0.125422, test_accuracy: 0.885548
train_loss: 0.088810, train_accuracy: 0.975250
test_loss: 0.125402, test_accurac

train_loss: 0.087122, train_accuracy: 0.978250
test_loss: 0.124306, test_accuracy: 0.885306
train_loss: 0.087103, train_accuracy: 0.978250
test_loss: 0.124297, test_accuracy: 0.885306
train_loss: 0.087083, train_accuracy: 0.978500
test_loss: 0.124288, test_accuracy: 0.885306
train_loss: 0.087064, train_accuracy: 0.978500
test_loss: 0.124279, test_accuracy: 0.885306
train_loss: 0.087044, train_accuracy: 0.978500
test_loss: 0.124270, test_accuracy: 0.885306
train_loss: 0.087025, train_accuracy: 0.978750
test_loss: 0.124261, test_accuracy: 0.885306
train_loss: 0.087006, train_accuracy: 0.979000
test_loss: 0.124253, test_accuracy: 0.885306
train_loss: 0.086986, train_accuracy: 0.979000
test_loss: 0.124244, test_accuracy: 0.885306
train_loss: 0.086967, train_accuracy: 0.979750
test_loss: 0.124236, test_accuracy: 0.885306
train_loss: 0.086948, train_accuracy: 0.980000
test_loss: 0.124227, test_accuracy: 0.885306
train_loss: 0.086929, train_accuracy: 0.980250
test_loss: 0.124219, test_accurac

train_loss: 0.085492, train_accuracy: 0.983250
test_loss: 0.123772, test_accuracy: 0.886275
train_loss: 0.085476, train_accuracy: 0.983250
test_loss: 0.123768, test_accuracy: 0.886275
train_loss: 0.085459, train_accuracy: 0.983250
test_loss: 0.123765, test_accuracy: 0.886275
train_loss: 0.085442, train_accuracy: 0.983250
test_loss: 0.123762, test_accuracy: 0.886275
train_loss: 0.085426, train_accuracy: 0.983250
test_loss: 0.123758, test_accuracy: 0.886275
train_loss: 0.085409, train_accuracy: 0.983250
test_loss: 0.123755, test_accuracy: 0.886275
train_loss: 0.085392, train_accuracy: 0.983250
test_loss: 0.123752, test_accuracy: 0.886275
train_loss: 0.085376, train_accuracy: 0.983250
test_loss: 0.123749, test_accuracy: 0.886275
train_loss: 0.085359, train_accuracy: 0.983250
test_loss: 0.123746, test_accuracy: 0.886275
train_loss: 0.085343, train_accuracy: 0.983250
test_loss: 0.123743, test_accuracy: 0.886760
train_loss: 0.085326, train_accuracy: 0.983250
test_loss: 0.123740, test_accurac

train_loss: 0.084093, train_accuracy: 0.994750
test_loss: 0.123605, test_accuracy: 0.886275
train_loss: 0.084079, train_accuracy: 0.994750
test_loss: 0.123604, test_accuracy: 0.886275
train_loss: 0.084065, train_accuracy: 0.995750
test_loss: 0.123604, test_accuracy: 0.886275
train_loss: 0.084050, train_accuracy: 0.996250
test_loss: 0.123603, test_accuracy: 0.886275
train_loss: 0.084036, train_accuracy: 0.996750
test_loss: 0.123603, test_accuracy: 0.886275
train_loss: 0.084022, train_accuracy: 0.996750
test_loss: 0.123602, test_accuracy: 0.886275
train_loss: 0.084007, train_accuracy: 0.996750
test_loss: 0.123602, test_accuracy: 0.886275
train_loss: 0.083993, train_accuracy: 0.997000
test_loss: 0.123601, test_accuracy: 0.886275
train_loss: 0.083979, train_accuracy: 0.997250
test_loss: 0.123601, test_accuracy: 0.886275
train_loss: 0.083965, train_accuracy: 0.997250
test_loss: 0.123600, test_accuracy: 0.886275
train_loss: 0.083951, train_accuracy: 0.997250
test_loss: 0.123600, test_accurac

train_loss: 0.082891, train_accuracy: 0.999250
test_loss: 0.123615, test_accuracy: 0.887003
train_loss: 0.082879, train_accuracy: 0.999250
test_loss: 0.123615, test_accuracy: 0.887003
train_loss: 0.082867, train_accuracy: 0.999250
test_loss: 0.123616, test_accuracy: 0.887003
train_loss: 0.082854, train_accuracy: 0.999250
test_loss: 0.123617, test_accuracy: 0.887003
train_loss: 0.082842, train_accuracy: 0.999250
test_loss: 0.123617, test_accuracy: 0.887245
train_loss: 0.082830, train_accuracy: 0.999250
test_loss: 0.123618, test_accuracy: 0.887245
train_loss: 0.082818, train_accuracy: 0.999250
test_loss: 0.123619, test_accuracy: 0.887245
train_loss: 0.082805, train_accuracy: 0.999250
test_loss: 0.123619, test_accuracy: 0.887245
train_loss: 0.082793, train_accuracy: 0.999250
test_loss: 0.123620, test_accuracy: 0.887245
train_loss: 0.082781, train_accuracy: 0.999250
test_loss: 0.123621, test_accuracy: 0.887245
train_loss: 0.082769, train_accuracy: 0.999250
test_loss: 0.123622, test_accurac

'Sequential training evaluation'

In [24]:
s

0.10442599370969949

In [25]:
logits_seq =  subnet_output(alpha_1, beta_1, X_) + subnet_output(alpha_2, beta_2, X_) 
#logits_init = subnet_output(alpha_1, beta_1, X_) + subnet_output(alpha_2, beta_2, X_)
loss_seq = tf.losses.mean_squared_error(labels=Y_, predictions=logits_seq)
accuracy_seq = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(Y_, axis=1), tf.argmax(logits_seq, axis=1)), dtype=tf.float32))

"""Initialize variables"""
init = tf.global_variables_initializer()
sess.run(init)

In [26]:
"""Sequential training"""
batch_size = 250

epoch_train_accuracy = []
epoch_test_accuracy = []
for epoch in range(700):
    #pbar = tqdm.tqdm(total=len(x_train), desc='sequential training phase')
    for i in range(0, len(x_train_seq), batch_size):
        x_batch = x_train_seq[i:i+batch_size]
        y_batch = y_train_seq[i:i+batch_size]
        if len(x_batch) != batch_size:
            break
        sess.run(E_2_seq, feed_dict={X_: x_batch, Y: y_batch})
        #pbar.update(n=len(x_batch))
    '''epoch evaluation'''
    [train_loss, train_accuracy] = sess.run([loss_seq, accuracy_seq], feed_dict={X_: x_train, Y: y_train})
    [test_loss, test_accuracy] = sess.run([loss_seq, accuracy_seq], feed_dict={X_: x_test, Y: y_test})
    print('train_loss: %f, train_accuracy: %f' % (train_loss, train_accuracy))
    print('test_loss: %f, test_accuracy: %f' % (test_loss, test_accuracy))
    epoch_train_accuracy.append(train_accuracy)
    epoch_test_accuracy.append(test_accuracy)
#sess.run(init_train_graph, feed_dict={X: x_train_init, Y: y_train_init})
print("Sequential training done")

"""Sequential training evaluation"""

train_loss: 0.133966, train_accuracy: 0.999250
test_loss: 0.176850, test_accuracy: 0.889185
train_loss: 0.134085, train_accuracy: 0.999250
test_loss: 0.176779, test_accuracy: 0.889185
train_loss: 0.134144, train_accuracy: 0.999250
test_loss: 0.176729, test_accuracy: 0.889185
train_loss: 0.134167, train_accuracy: 0.999250
test_loss: 0.176691, test_accuracy: 0.889185
train_loss: 0.134169, train_accuracy: 0.999250
test_loss: 0.176661, test_accuracy: 0.889185
train_loss: 0.134159, train_accuracy: 0.999250
test_loss: 0.176637, test_accuracy: 0.889185
train_loss: 0.134142, train_accuracy: 0.999250
test_loss: 0.176617, test_accuracy: 0.889185
train_loss: 0.134121, train_accuracy: 0.999250
test_loss: 0.176599, test_accuracy: 0.889185
train_loss: 0.134098, train_accuracy: 0.999250
test_loss: 0.176583, test_accuracy: 0.889185
train_loss: 0.134074, train_accuracy: 0.999250
test_loss: 0.176569, test_accuracy: 0.889185
train_loss: 0.134050, train_accuracy: 0.999250
test_loss: 0.176556, test_accurac

train_loss: 0.133267, train_accuracy: 0.999250
test_loss: 0.177169, test_accuracy: 0.890155
train_loss: 0.133263, train_accuracy: 0.999250
test_loss: 0.177184, test_accuracy: 0.890155
train_loss: 0.133259, train_accuracy: 0.999250
test_loss: 0.177199, test_accuracy: 0.890155
train_loss: 0.133254, train_accuracy: 0.999250
test_loss: 0.177214, test_accuracy: 0.890155
train_loss: 0.133250, train_accuracy: 0.999250
test_loss: 0.177229, test_accuracy: 0.890155
train_loss: 0.133246, train_accuracy: 0.999250
test_loss: 0.177244, test_accuracy: 0.890155
train_loss: 0.133242, train_accuracy: 0.999250
test_loss: 0.177259, test_accuracy: 0.890155
train_loss: 0.133238, train_accuracy: 0.999250
test_loss: 0.177274, test_accuracy: 0.890155
train_loss: 0.133234, train_accuracy: 0.999250
test_loss: 0.177289, test_accuracy: 0.890398
train_loss: 0.133230, train_accuracy: 0.999250
test_loss: 0.177304, test_accuracy: 0.890398
train_loss: 0.133226, train_accuracy: 0.999250
test_loss: 0.177319, test_accurac

train_loss: 0.132966, train_accuracy: 0.999250
test_loss: 0.178536, test_accuracy: 0.891610
train_loss: 0.132964, train_accuracy: 0.999250
test_loss: 0.178550, test_accuracy: 0.891610
train_loss: 0.132961, train_accuracy: 0.999250
test_loss: 0.178565, test_accuracy: 0.891610
train_loss: 0.132958, train_accuracy: 0.999250
test_loss: 0.178579, test_accuracy: 0.891610
train_loss: 0.132956, train_accuracy: 0.999250
test_loss: 0.178593, test_accuracy: 0.891610
train_loss: 0.132953, train_accuracy: 0.999250
test_loss: 0.178607, test_accuracy: 0.891610
train_loss: 0.132950, train_accuracy: 0.999250
test_loss: 0.178622, test_accuracy: 0.891610
train_loss: 0.132948, train_accuracy: 0.999250
test_loss: 0.178636, test_accuracy: 0.891610
train_loss: 0.132945, train_accuracy: 0.999250
test_loss: 0.178650, test_accuracy: 0.891610
train_loss: 0.132942, train_accuracy: 0.999250
test_loss: 0.178664, test_accuracy: 0.891610
train_loss: 0.132939, train_accuracy: 0.999250
test_loss: 0.178679, test_accurac

train_loss: 0.132747, train_accuracy: 0.999250
test_loss: 0.179746, test_accuracy: 0.895490
train_loss: 0.132745, train_accuracy: 0.999250
test_loss: 0.179759, test_accuracy: 0.895490
train_loss: 0.132743, train_accuracy: 0.999250
test_loss: 0.179771, test_accuracy: 0.895490
train_loss: 0.132741, train_accuracy: 0.999250
test_loss: 0.179784, test_accuracy: 0.895490
train_loss: 0.132739, train_accuracy: 0.999250
test_loss: 0.179796, test_accuracy: 0.895490
train_loss: 0.132737, train_accuracy: 0.999250
test_loss: 0.179809, test_accuracy: 0.895490
train_loss: 0.132735, train_accuracy: 0.999250
test_loss: 0.179821, test_accuracy: 0.895490
train_loss: 0.132732, train_accuracy: 0.999250
test_loss: 0.179833, test_accuracy: 0.895490
train_loss: 0.132730, train_accuracy: 0.999250
test_loss: 0.179846, test_accuracy: 0.895490
train_loss: 0.132728, train_accuracy: 0.999250
test_loss: 0.179858, test_accuracy: 0.895490
train_loss: 0.132726, train_accuracy: 0.999250
test_loss: 0.179871, test_accurac

train_loss: 0.132577, train_accuracy: 0.999250
test_loss: 0.180801, test_accuracy: 0.895732
train_loss: 0.132575, train_accuracy: 0.999250
test_loss: 0.180812, test_accuracy: 0.895732
train_loss: 0.132574, train_accuracy: 0.999250
test_loss: 0.180823, test_accuracy: 0.895732
train_loss: 0.132572, train_accuracy: 0.999250
test_loss: 0.180834, test_accuracy: 0.895732
train_loss: 0.132570, train_accuracy: 0.999250
test_loss: 0.180845, test_accuracy: 0.895975
train_loss: 0.132569, train_accuracy: 0.999250
test_loss: 0.180856, test_accuracy: 0.896217
train_loss: 0.132567, train_accuracy: 0.999250
test_loss: 0.180867, test_accuracy: 0.896217
train_loss: 0.132566, train_accuracy: 0.999250
test_loss: 0.180878, test_accuracy: 0.896217
train_loss: 0.132564, train_accuracy: 0.999250
test_loss: 0.180888, test_accuracy: 0.896217
train_loss: 0.132562, train_accuracy: 0.999250
test_loss: 0.180899, test_accuracy: 0.896217
train_loss: 0.132561, train_accuracy: 0.999250
test_loss: 0.180910, test_accurac

train_loss: 0.132448, train_accuracy: 0.999250
test_loss: 0.181726, test_accuracy: 0.895005
train_loss: 0.132446, train_accuracy: 0.999250
test_loss: 0.181735, test_accuracy: 0.895005
train_loss: 0.132445, train_accuracy: 0.999250
test_loss: 0.181745, test_accuracy: 0.895005
train_loss: 0.132444, train_accuracy: 0.999250
test_loss: 0.181755, test_accuracy: 0.895005
train_loss: 0.132443, train_accuracy: 0.999250
test_loss: 0.181764, test_accuracy: 0.895005
train_loss: 0.132442, train_accuracy: 0.999250
test_loss: 0.181774, test_accuracy: 0.895005
train_loss: 0.132440, train_accuracy: 0.999250
test_loss: 0.181783, test_accuracy: 0.895005
train_loss: 0.132439, train_accuracy: 0.999250
test_loss: 0.181793, test_accuracy: 0.895005
train_loss: 0.132438, train_accuracy: 0.999250
test_loss: 0.181802, test_accuracy: 0.895005
train_loss: 0.132437, train_accuracy: 0.999250
test_loss: 0.181812, test_accuracy: 0.895005
train_loss: 0.132435, train_accuracy: 0.999250
test_loss: 0.181821, test_accurac

train_loss: 0.132351, train_accuracy: 0.999250
test_loss: 0.182540, test_accuracy: 0.894035
train_loss: 0.132350, train_accuracy: 0.999250
test_loss: 0.182548, test_accuracy: 0.894035
train_loss: 0.132349, train_accuracy: 0.999250
test_loss: 0.182557, test_accuracy: 0.894035
train_loss: 0.132348, train_accuracy: 0.999250
test_loss: 0.182565, test_accuracy: 0.894035
train_loss: 0.132347, train_accuracy: 0.999250
test_loss: 0.182574, test_accuracy: 0.894035
train_loss: 0.132347, train_accuracy: 0.999250
test_loss: 0.182582, test_accuracy: 0.894035
train_loss: 0.132346, train_accuracy: 0.999250
test_loss: 0.182590, test_accuracy: 0.894035
train_loss: 0.132345, train_accuracy: 0.999250
test_loss: 0.182599, test_accuracy: 0.894035
train_loss: 0.132344, train_accuracy: 0.999250
test_loss: 0.182607, test_accuracy: 0.894035
train_loss: 0.132343, train_accuracy: 0.999250
test_loss: 0.182616, test_accuracy: 0.894035
train_loss: 0.132342, train_accuracy: 0.999250
test_loss: 0.182624, test_accurac

train_loss: 0.132278, train_accuracy: 0.999250
test_loss: 0.183290, test_accuracy: 0.893065
train_loss: 0.132278, train_accuracy: 0.999250
test_loss: 0.183297, test_accuracy: 0.893065
train_loss: 0.132277, train_accuracy: 0.999250
test_loss: 0.183305, test_accuracy: 0.893065
train_loss: 0.132276, train_accuracy: 0.999250
test_loss: 0.183312, test_accuracy: 0.893065
train_loss: 0.132275, train_accuracy: 0.999250
test_loss: 0.183320, test_accuracy: 0.893065
train_loss: 0.132275, train_accuracy: 0.999250
test_loss: 0.183327, test_accuracy: 0.893065
train_loss: 0.132274, train_accuracy: 0.999250
test_loss: 0.183334, test_accuracy: 0.893065
train_loss: 0.132273, train_accuracy: 0.999250
test_loss: 0.183342, test_accuracy: 0.893065
train_loss: 0.132273, train_accuracy: 0.999250
test_loss: 0.183349, test_accuracy: 0.893065
train_loss: 0.132272, train_accuracy: 0.999250
test_loss: 0.183357, test_accuracy: 0.893065
train_loss: 0.132272, train_accuracy: 0.999250
test_loss: 0.183364, test_accurac

'Sequential training evaluation'

In [27]:
"""Evaluate initial-training graph"""
logits_seq =  subnet_output(alpha_1, beta_1, X_) 
#logits_init = subnet_output(alpha_1, beta_1, X_) + subnet_output(alpha_2, beta_2, X_)
loss_seq = tf.losses.mean_squared_error(labels=Y_, predictions=logits_seq)
accuracy_seq = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(Y_, axis=1), tf.argmax(logits_seq, axis=1)), dtype=tf.float32))

In [28]:
"""Initialize variables"""
init = tf.global_variables_initializer()
sess.run(init)

In [29]:
"""Sequential training"""
batch_size = 250

epoch_train_accuracy = []
epoch_test_accuracy = []
for epoch in range(40):
    #pbar = tqdm.tqdm(total=len(x_train), desc='sequential training phase')
    for i in range(0, len(x_train_seq), batch_size):
        x_batch = x_train_seq[i:i+batch_size]
        y_batch = y_train_seq[i:i+batch_size]
        if len(x_batch) != batch_size:
            break
        sess.run(E_1_seq, feed_dict={X_: x_batch, Y: y_batch})
        #pbar.update(n=len(x_batch))
    '''epoch evaluation'''
    [train_loss, train_accuracy] = sess.run([loss_seq, accuracy_seq], feed_dict={X_: x_train, Y: y_train})
    [test_loss, test_accuracy] = sess.run([loss_seq, accuracy_seq], feed_dict={X_: x_test, Y: y_test})
    print('train_loss: %f, train_accuracy: %f' % (train_loss, train_accuracy))
    print('test_loss: %f, test_accuracy: %f' % (test_loss, test_accuracy))
    epoch_train_accuracy.append(train_accuracy)
    epoch_test_accuracy.append(test_accuracy)
#sess.run(init_train_graph, feed_dict={X: x_train_init, Y: y_train_init})
print("Sequential training done")

"""Sequential training evaluation"""

train_loss: 0.156346, train_accuracy: 0.827250
test_loss: 0.356628, test_accuracy: 0.217992
train_loss: 0.139132, train_accuracy: 0.827250
test_loss: 0.334477, test_accuracy: 0.217992
train_loss: 0.127112, train_accuracy: 0.827250
test_loss: 0.310640, test_accuracy: 0.217992
train_loss: 0.115784, train_accuracy: 0.887250
test_loss: 0.287823, test_accuracy: 0.417071
train_loss: 0.105274, train_accuracy: 0.965750
test_loss: 0.266905, test_accuracy: 0.475994
train_loss: 0.096003, train_accuracy: 0.998500
test_loss: 0.248398, test_accuracy: 0.500970
train_loss: 0.088182, train_accuracy: 0.988000
test_loss: 0.232520, test_accuracy: 0.548739
train_loss: 0.081778, train_accuracy: 0.982250
test_loss: 0.219195, test_accuracy: 0.600630
train_loss: 0.076612, train_accuracy: 0.975750
test_loss: 0.208148, test_accuracy: 0.650097
train_loss: 0.072458, train_accuracy: 0.975250
test_loss: 0.199027, test_accuracy: 0.685015
train_loss: 0.069101, train_accuracy: 0.975250
test_loss: 0.191482, test_accurac

'Sequential training evaluation'

In [30]:
logits_ = subnet_output(alpha_1, beta_1, X_) 
logits__ = sess.run(logits_, feed_dict={X: [x_test[4000]]})
print(logits__)
print(np.argmax(logits__))
print(y_test[4000])
plt.imshow(x_test[4000])

NameError: name 'X' is not defined

In [ ]:
def visualize_alpha(alpha, size):
    tmp = sess.run(alpha)
    plt.figure(figsize=(10,4))
    for i in range(2):
        for j in range(5):
            plt.subplot(2,5,i*5+j+1)
            plt.imshow(np.reshape(tmp[:,i*5+j], [size,size]))

def visualize_beta(beta):
    tmp = sess.run(beta)
    plt.figure(figsize=(4,4))
    plt.imshow(tmp)
    
            
"""visualize subnet nodes"""            
visualize_alpha(alpha_1, 10)
visualize_beta(beta_1)
visualize_alpha(alpha_2, 10)
visualize_beta(beta_2)

In [ ]:
from ml_elm import ML_ELM

mlelm1 = ML_ELM(input_size=input_size, output_size=output_size, name='mlelm1')


In [ ]:
sess.run(tf.math.sin(tf.constant([0.9])))